In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from collections import Counter

print("\U0001F680 Starting the program...")

# Load Preprocessed Dataset
print("\U0001F4C2 Loading preprocessed data...")
X = np.load("Processed_Data/images.npy", mmap_mode="r")
y = np.load("Processed_Data/labels.npy", mmap_mode="r")

# Check dataset balance
class_counts = Counter(y)
print("Class Distribution:", class_counts)

# Define Data Generator
def data_generator():
    for x, label in zip(X, y):
        yield x.flatten(), label  # Flatten images for fully connected layers

# Create TensorFlow Dataset
print("\U0001F4E1 Creating TensorFlow dataset...")
dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_signature=(
        tf.TensorSpec(shape=(128*128*3,), dtype=tf.float32),  # Flattened input
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)

# Split into Training & Validation
total_size = X.shape[0]
train_size = int(0.8 * total_size)
val_size = total_size - train_size

train_ds = dataset.take(train_size).shuffle(1000).batch(24).prefetch(tf.data.AUTOTUNE).repeat()
val_ds = dataset.skip(train_size).take(val_size).batch(24).prefetch(tf.data.AUTOTUNE)



# Define Data Augmentation
print("\U0001F3A8 Setting up data augmentation...")
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2)
])


# Define Fully Connected Model (Based on Screenshot)
print("\U0001F6E0 Building the Fully Connected Model...")
model = models.Sequential([
    layers.Input(shape=(128 * 128 * 3,)),
    layers.Reshape((128, 128, 3)),  
    data_augmentation,
    layers.Flatten(),  

    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.03)),
    layers.BatchNormalization(),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.03)),
    
    layers.Dense(2, activation='softmax')  
])


# Compile Model
print("\u2699\ufe0f Compiling the model...")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.03),  # Match screenshot's learning rate
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train Model for ONE Epoch Only
EPOCHS = 3
steps_per_epoch = train_size // 24  # Ensure full batches

print("\U0001F6A6 Training for 1 epoch...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch
)

# Extract & Print First Epoch Metrics
first_epoch_loss = history.history["loss"][0]
first_epoch_accuracy = history.history["accuracy"][0]
val_loss = history.history["val_loss"][0]
val_accuracy = history.history["val_accuracy"][0]

print(f"\n🔹 First Epoch Results:")
print(f"   - Training Loss: {first_epoch_loss:.4f}")
print(f"   - Training Accuracy: {first_epoch_accuracy:.4%}")
print(f"   - Validation Loss: {val_loss:.4f}")
print(f"   - Validation Accuracy: {val_accuracy:.4%}")

# Check for Overfitting
if first_epoch_accuracy > 0.98 and (first_epoch_accuracy - val_accuracy) > 0.05:
    print("\U000026A0 Warning: Possible overfitting detected! Training accuracy is much higher than validation accuracy.")

#  Save the Model
model.save("best_model.keras")
print("\U0001F4BE Model saved as 'best_model.keras'!")

print("\U0001F389 First Epoch Completed!")
# model.predict()
